# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.



nohup eva_server >> eva.log 2>&1 &
('YOLOv5 🚀 2023-4-12 Python-3.9.2 torch-2.0.0+cu117 CPU\n', '\n', 'Downloading https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt to yolov5s.pt...\n')
('\n', '  0%|          | 0.00/14.1M [00:00<?, ?B/s]\n', ' 10%|█         | 1.48M/14.1M [00:00<00:00, 15.5MB/s]\n')
(' 62%|██████▏   | 8.70M/14.1M [00:00<00:00, 50.8MB/s]\n', '100%|██████████| 14.1M/14.1M [00:00<00:00, 55.6MB/s]\n', '\n')
('Fusing layers... \n', 'YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients\n', 'Adding AutoShape... \n')
('04-12-2023 03:02:25 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF FastRCNNObjectDetector already exists, nothing added.\n', '04-12-2023 03:02:25 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF ArrayCount already exists, nothing added.\n', '04-12-2023 03:02:25 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF Crop already exists, nothing added.\n')
('04-12-2023 03:02:25 WARNING[crea

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [3]:
# sample dataset of 2 videos
!wget -nc https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip

# unzip
!unzip -n bddtest.zip

--2023-04-12 03:05:02--  https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... 

connected.


HTTP request sent, awaiting response... 

302 Found
Location: /s/raw/kg1q69ijbhjfecw/bddtest.zip [following]
--2023-04-12 03:05:03--  https://www.dropbox.com/s/raw/kg1q69ijbhjfecw/bddtest.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 

302 Found
Location: https://ucbbf1a0aa64c6eaafec306b3b75.dl.dropboxusercontent.com/cd/0/inline/B6AbG7jYTfdwgJRvc4mj3cnoh583_LPIcYHVi1QF3M1aHAftxBG9yUKaJj90mJl95XNCcU4fedwxINs0Rq7Aqt1Sj4cf3MBb3Y67snM-Szik1RJOVbpuSDmA_e0A-3_Fyuds4_pyQB18Wl69spceSWSMlVe5DgYoMxDPV3tlF7fpoA/file# [following]
--2023-04-12 03:05:03--  https://ucbbf1a0aa64c6eaafec306b3b75.dl.dropboxusercontent.com/cd/0/inline/B6AbG7jYTfdwgJRvc4mj3cnoh583_LPIcYHVi1QF3M1aHAftxBG9yUKaJj90mJl95XNCcU4fedwxINs0Rq7Aqt1Sj4cf3MBb3Y67snM-Szik1RJOVbpuSDmA_e0A-3_Fyuds4_pyQB18Wl69spceSWSMlVe5DgYoMxDPV3tlF7fpoA/file
Resolving ucbbf1a0aa64c6eaafec306b3b75.dl.dropboxusercontent.com (ucbbf1a0aa64c6eaafec306b3b75.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:601c:15::a27d:60f
Connecting to ucbbf1a0aa64c6eaafec306b3b75.dl.dropboxusercontent.com (ucbbf1a0aa64c6eaafec306b3b75.dl.dropboxusercontent.com)|162.125.3.15|:443... connected.


...

bddtest.zip          89%[================>   ] 140.19M  33.4MB/s    eta 1s     

bddtest.zip          94%[=================>  ] 148.01M  33.6MB/s    eta 1s     

bddtest.zip          98%[==================> ] 155.37M  33.3MB/s    eta 1s     

bddtest.zip         100%[===================>] 157.37M  33.4MB/s    in 5.1s    

2023-04-12 03:05:10 (31.1 MB/s) - ‘bddtest.zip’ saved [165016842/165016842]



/bin/bash: line 1: unzip: command not found


## Load 2 BDD videos

In [4]:
cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load VIDEO failed: encountered unexpected error Load VIDEO failed due to no valid files found on path bddtest/videos/bddtest_00a2e3ca5c856cde.mp4
@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load VIDEO failed: encountered unexpected error Load VIDEO failed due to no valid files found on path bddtest/videos/bddtest_00a395fed60c0b47.mp4


## Create a BDD Meta-data Table

In [5]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.1129953239999395


## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [6]:
cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: The dataset file could not be found. Please verify that the file exists in the specified path.
@status: ResponseStatus.FAIL
@batch: 
 None
@error: The dataset file could not be found. Please verify that the file exists in the specified path.


## Run a Query over the Video dataset


In [7]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) 
                  FROM bddtest_1 
                  WHERE id < 20;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: bddtest_1 does not exist. Create the table using CREATE TABLE.


## Run a query over the Meta-data Table

In [8]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.020194305999893913
